In [1]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import  time 
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.nn.modules.loss import _Loss 
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *

c:\users\hello\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [4]:
###
def split4(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(F.interpolate(img, scale_factor=1))

    output.append(img)
    return output

In [5]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

In [6]:
training_x=[]#train 1 tomorrow

path='./data/input/'#要改
pathdepth='./data/depth/'#要改

path_list = os.listdir(path)
# path_list.sort(key=lambda x:int(x.split('.')[0]))
path_list.sort(key=lambda x:(x.split('.')[0]))

testequal = []
i= 0

for item in path_list:
    i+=1
    if i==199:
        testequal.append(item)
        i=0
    
    impath=path+item
#     print("开始处理"+impath)
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    
    imgd= cv2.imread(pathdepth+item.split(".")[0]+".png",0)
    imgd= cv2.resize(imgd,(256,256)).reshape((256,256,1))
    imgd = (imgd  / np.max(imgd))* 255
    imgx = np.concatenate((imgx, imgd), axis=2)
    
    training_x.append(imgx)   

X_train = []

for features in training_x:
    X_train.append(features)
    
X_train = np.array(X_train)

In [7]:
training_x=0

In [8]:
# from os import path

# for item in path_list:
#     if(not path.exists(pathdepth+item.split(".")[0]+".png")):
#         print(item)

In [9]:
# imgx= cv2.imread("./data/input/0001.png")
# imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
# imgx=cv2.resize(imgx,(256,256))

# imgd= cv2.imread("./data/depth/0001.png",0)
# imgd= cv2.resize(imgd,(256,256)).reshape((256,256,1))
# imgd = (imgd  / np.max(imgd))* 255

# # print(imgx.shape,imgd.shape)
# pp=np.concatenate((imgx, imgd), axis=2)

In [10]:
# X_train

In [11]:
X_train.shape

(11170, 256, 256, 4)

In [12]:
X_train=X_train.astype(dtype)
X_train= torch.from_numpy(X_train)
X_train=X_train.permute(0,3,1,2)

In [13]:
X_train=X_train/255.0
X_train.shape

torch.Size([11170, 4, 256, 256])

In [14]:
training_y=[]
path='./data/GT/'#要改
path_list = os.listdir(path)
# path_list.sort(key=lambda x:int(x.split('.')[0]))
path_list.sort(key=lambda x:(x.split('.')[0]))


i= 0
j=0
for item in path_list:
    i+=1
    if i==199:
        if(testequal[j]!= item):
            print("Error")
        i=0
        j+=1
    
    impath=path+item
#     print("开始处理"+impath)
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    training_y.append(imgx)
    
    
    

y_train = []

for features in training_y:
    y_train.append(features)
    

    
y_train = np.array(y_train)


In [15]:
training_y=0
testequal=0

In [16]:
y_train=y_train.astype(dtype)
y_train= torch.from_numpy(y_train)
y_train=y_train.permute(0,3,1,2)

In [17]:
y_train=y_train/255.0
y_train.shape

torch.Size([11170, 3, 256, 256])

In [18]:
test_x=[]
path='./test/input/'#要改
pathdepth = "./test/depth/"
path_list = os.listdir(path)
# path_list.sort(key=lambda x:int(x.split('.')[0]))
path_list.sort(key=lambda x:(x.split('.')[0]))

for item in path_list:
    impath=path+item
    #print("开始处理"+impath)
    imgx= cv2.imread(path+item)
    if imgx is None:
        print(path+item)
        continue
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    
    imgd= cv2.imread(pathdepth+item.split(".")[0]+".png",0)
    imgd= cv2.resize(imgd,(256,256)).reshape((256,256,1))
    imgd = (imgd  / np.max(imgd))* 255
    imgx = np.concatenate((imgx, imgd), axis=2)
    
    test_x.append(imgx)
    
    
    

x_test = []

for features in test_x:
    x_test.append(features)
    

    
x_test = np.array(x_test)

In [19]:
test_x=0

In [20]:
x_test=x_test.astype(dtype)
x_test= torch.from_numpy(x_test)
x_test=x_test.permute(0,3,1,2)

In [21]:
x_test=x_test/255.0

In [22]:
x_test.shape

torch.Size([1214, 4, 256, 256])

In [23]:
test_Y=[]
path='./test/GT/'#要改
path_list = os.listdir(path)
# path_list.sort(key=lambda x:int(x.split('.')[0]))
path_list.sort(key=lambda x:(x.split('.')[0]))

for item in path_list:
    impath=path+item
    #print("开始处理"+impath)
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    test_Y.append(imgx)
    
    
    

Y_test = []

for features in test_Y:
    Y_test.append(features)
    

    
Y_test = np.array(Y_test)
#X_train = np.array(X_train)

In [24]:
test_Y=0

In [25]:
Y_test=Y_test.astype(dtype)
Y_test= torch.from_numpy(Y_test)
Y_test=Y_test.permute(0,3,1,2)

In [26]:
Y_test=Y_test/255.0

In [27]:
Y_test.shape

torch.Size([1214, 3, 256, 256])

In [28]:
import torch.utils.data as dataf
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


dataset = dataf.TensorDataset(X_train,y_train)
loader = dataf.DataLoader(dataset, batch_size=1, shuffle=True,num_workers=4)

In [29]:
criterion_GAN=nn.MSELoss(size_average=False)
criterion_pixelwise=nn.MSELoss(size_average=False)
MSE = nn.MSELoss(size_average=False)
SSIM = pytorch_ssim.SSIM()
L_vgg = VGG19_PercepLoss()
L_lab=lab_Loss()
L_lch=lch_Loss()

criterion_pixelwise.cuda()
L_vgg.cuda()
MSE.cuda()
SSIM.cuda()
criterion_GAN.cuda()
L_lab.cuda()
L_lch.cuda()

c:\users\hello\appdata\local\programs\python\python38\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
c:\users\hello\appdata\local\programs\python\python38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\users\hello\appdata\local\programs\python\python38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


lch_Loss()

In [30]:
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel=0.1
lambda_lab=0.001
lambda_lch=1
lambda_con = 100
lambda_ssim=100

In [31]:
# Calculate output of image discriminator (PatchGAN)
patch = (1, 256 // 2 ** 5, 256// 2 ** 5)

In [32]:
# Discriminator()

In [33]:
# Initialize generator and discriminator
generator = Generator().cuda()
discriminator = Discriminator().cuda()

In [34]:
# 如果有保存的模型，则加载模型，并在其基础上继续训练
use_pretrain=True
if use_pretrain:
    # Load pretrained models
    start_epoch=160
    generator.load_state_dict(torch.load("saved_models/G/generator_%d.pth" % (start_epoch)))
    discriminator.load_state_dict(torch.load("saved_models/D/discriminator_%d.pth" % (start_epoch)))
    print('successfully loading epoch {} 成功！'.format(start_epoch))
else:
    start_epoch = 0
    print('No pretrain model found, training will start from scratch！')

successfully loading epoch 160 成功！


In [43]:
def sample_images(batches_done):
    """Saves a generated sample from the validation set"""
    generator.eval()
    i=random.randrange(1,90)
    real_A = Variable(x_test[i,:,:,:]).cuda()
    real_B = Variable(Y_test[i,:,:,:]).cuda()
    real_A=real_A.unsqueeze(0)
    real_B=real_B.unsqueeze(0)
    fake_B = generator(real_A)
    #print(fake_B.shape)
    imgx=fake_B[3].data
    imgy=real_B.data
    x=imgx[:,:,:,:]
    y=imgy[:,:,:,:]
    img_sample = torch.cat((x,y), -2)
    save_image(img_sample, "images/%s/%s.png" % ('results', batches_done), nrow=5, normalize=True)#要改

In [44]:
from torch.optim import lr_scheduler
LR=0.0005

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=LR,  betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LR,  betas=(0.5, 0.999))
scheduler_G=lr_scheduler.StepLR(optimizer_G,step_size=40,gamma=0.8)
scheduler_D=lr_scheduler.StepLR(optimizer_D,step_size=40,gamma=0.8)

In [45]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [47]:
# ----------
#  Training
# ----------
import time as time
import datetime
import sys
from torchvision.utils import save_image
import csv
import random


f1 = open('psnr.csv','w',encoding='utf-8')#要改
csv_writer1 = csv.writer(f1)
f2 = open('SSIM.csv','w',encoding='utf-8')#要改
csv_writer2 = csv.writer(f2)

checkpoint_interval=1
epochs=start_epoch
n_epochs=300
sample_interval=1000

# ingnored when opt.mode=='S'
psnr_list = [] 
prev_time = time.time()

for epoch in range(epochs,n_epochs):
    for i, batch in enumerate(loader):

        # Model inputs
        real_A = Variable(batch[0]).cuda() 
        real_B = Variable(batch[1]).cuda()
        #real_A1=Tensor(np.array(split(real_A)).astype(dtype))
        #real_B1=Tensor(np.array(split(real_B)).astype(dtype))
        real_A1=split(real_A[:,1:,:,:])
        real_B1=split(real_B)
        #print(len(real_A1))
        #print(len(real_B1))
        #real_A1= torch.tensor([item.cpu().detach().numpy() for item in real_A1]).cuda() 
        #real_B1= torch.tensor([item.cpu().detach().numpy() for item in real_B1]).cuda() 


        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *patch))), requires_grad=False)#全1
        fake = Variable(Tensor(np.zeros((real_A.size(0), *patch))), requires_grad=False)#全0

        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()

        # GAN loss
        fake_B = generator(real_A)
        
#         print(len(fake_B))
#         print(fake_B[0].shape)
#         print(fake_B[1].shape)
#         print(fake_B[2].shape)
#         print(fake_B[3].shape)
        
#         print(len(real_A1))
#         print(real_A1[0].shape)
#         print(real_A1[1].shape)
#         print(real_A1[2].shape)
#         print(real_A1[3].shape)
        
        #fake_B1 = list(map(lambda x: x.detach(), fake_B))
        pred_fake = discriminator(fake_B, real_A1)
        loss_GAN = criterion_GAN(pred_fake, valid)
        # Pixel-wise loss
#         print(len(fake_B))
#         print(fake_B[0].shape)
#         print(real_B1[0].shape)

#         fake3 = fake_B.copy()
#         fake_B[0] = fake_B[0][:,1:,:,:]
#         fake_B[1] = fake_B[1][:,1:,:,:]
#         fake_B[2] = fake_B[2][:,1:,:,:]
#         fake_B[3] = fake_B[3][:,1:,:,:]

#         print(len(fake_B))
#         print(fake_B[0].shape)
#         print(fake_B[1].shape)
#         print(fake_B[2].shape)
#         print(fake_B[3].shape)
        
#         print(len(real_B1))
#         print(real_B1[0].shape)
#         print(real_B1[1].shape)
#         print(real_B1[2].shape)
#         print(real_B1[3].shape)

        loss_pixel = (criterion_pixelwise(fake_B[0], real_B1[0])+\
                      criterion_pixelwise(fake_B[1], real_B1[1])+\
                      criterion_pixelwise(fake_B[2], real_B1[2])+\
                      criterion_pixelwise(fake_B[3], real_B1[3]))/4.0
        loss_ssim=   -(SSIM(fake_B[0], real_B1[0])+\
                      SSIM(fake_B[1], real_B1[1])+\
                      SSIM(fake_B[2], real_B1[2])+\
                      SSIM(fake_B[3], real_B1[3]))/4.0
        ssim_value = - loss_ssim.item()
        loss_con = (L_vgg(fake_B[0], real_B1[0])+\
                    L_vgg(fake_B[1], real_B1[1])+\
                    L_vgg(fake_B[2], real_B1[2])+\
                    L_vgg(fake_B[3], real_B1[3]))/4.0
        loss_lab = (L_lab(fake_B[0], real_B1[0])+\
                    L_lab(fake_B[1], real_B1[1])+\
                    L_lab(fake_B[2], real_B1[2])+\
                    L_lab(fake_B[3], real_B1[3]))/4.0
        loss_lch = (L_lch(fake_B[0], real_B1[0])+\
                    L_lch(fake_B[1], real_B1[1])+\
                    L_lch(fake_B[2], real_B1[2])+\
                    L_lch(fake_B[3], real_B1[3]))/4.0     
        

        # Total loss
        loss_G = (loss_GAN + lambda_pixel * loss_pixel+lambda_ssim*loss_ssim+\
        lambda_con*loss_con+lambda_lab*loss_lab+lambda_lch*loss_lch)

        loss_G.backward(retain_graph=True)

        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()
        
#         fake_B = fake3
#         print(len(fake_B))
#         print(real_B1[0].shape)
#         print(real_B1[1].shape)
#         print(real_B1[2].shape)
#         print(real_B1[3].shape)
        
#         print(len(real_A1))
#         print(real_A1[0].shape)
#         print(real_A1[1].shape)
#         print(real_A1[2].shape)
#         print(real_A1[3].shape)
        # Real loss
        pred_real = discriminator(real_B1, real_A1)
        loss_real = criterion_GAN(pred_real, valid)

        # Fake loss
        fake_B[0]=fake_B[0].detach()
        fake_B[1]=fake_B[1].detach()
        fake_B[2]=fake_B[2].detach()
        fake_B[3]=fake_B[3].detach()
        pred_fake1 = discriminator(fake_B, real_A1)
        loss_fake = criterion_GAN(pred_fake1, fake)

        # Total loss
        loss_D = 0.5 * (loss_real + loss_fake)
        #loss_D=loss_real

        loss_D.backward(retain_graph=True)
        optimizer_D.step()

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(loader) + i
        batches_left = n_epochs * len(loader) - batches_done
        out_train= torch.clamp(fake_B[3], 0., 1.) 
        psnr_train = batch_PSNR(out_train,real_B, 1.)
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d][PSNR: %f] [SSIM: %f] [D loss: %f] [G loss: %f],[lab: %f],[lch: %f], [pixel: %f],[VGG_loss: %f], [adv: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(loader),
                psnr_train,
                ssim_value,
                loss_D.item(),
                loss_G.item(),
                0.001*loss_lab.item(),
                1*loss_lch.item(),
                0.1*loss_pixel.item(),
                100*loss_con.item(),
                loss_GAN.item(),
                
                
                time_left,
            )
        )
        

        # If at sample interval save image
        if batches_done % sample_interval == 0:
            sample_images(batches_done)
            csv_writer1.writerow([str(psnr_train)])
            csv_writer2.writerow([str(ssim_value)])
            
            
            
            
    scheduler_G.step()
    scheduler_D.step()
    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        # Save model checkpoints
        torch.save(generator.state_dict(), "saved_models/G/generator_%d.pth" % (epoch))
        torch.save(discriminator.state_dict(), "saved_models/D/discriminator_%d.pth" % (epoch))

[Epoch 170/300] [Batch 66/11170][PSNR: 22.071240] [SSIM: 0.774581] [D loss: 0.016711] [G loss: 104.821472],[lab: 44.235352],[lch: 11.941713], [pixel: 41.190018],[VGG_loss: 21.668670], [adv: 63.243813] ETA: 2 days, 9:44:39.91718958015015

KeyboardInterrupt: 

In [39]:
# import net.Ushape_Trans  as un
# import importlib
# importlib.reload(un)
# un.Generator().out_ch


In [40]:
# generator = un.Generator().cuda()
# discriminator = un.Discriminator().cuda()


In [41]:
# from skimage.metrics import peak_signal_noise_ratio
 

In [42]:
# Generator().in_ch

In [43]:
Generator().num_layers

AttributeError: 'Generator' object has no attribute 'num_layers'

In [ ]:
fake_B[0][:,1:,:,:].shape

In [ ]:
real_B1[0].shape

In [ ]:
Y_test.shape
import matplotlib.pyplot as plt


In [ ]:
Y_test[0].permute(1,2,0).shape